## GoogLeNet

In [1]:
!nvidia-smi

Mon Mar  7 20:06:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:09.0 Off |                    0 |
| N/A   54C    P0    56W / 300W |   9462MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import paddle
import matplotlib.pyplot as plt
import matplotlib.image as mping
import numpy as np

paddle.__version__, paddle.device.get_device()

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


('2.2.2', 'gpu:0')

### Data loading

In [3]:
import paddle.vision.transforms as T
from paddle.vision.datasets import FashionMNIST

# loading and normalization
# resize the shape to 96x96
transform = [T.Resize(size=96), T.Normalize(mean=[127.5], std=[127.5])]
transform = T.Compose(transform)

# constructing traning set and test set
fashionmnist_train = FashionMNIST(mode='train', transform=transform)
fashionmnist_test = FashionMNIST(mode='test', transform=transform)

In [4]:
# nums of train set and test set
len(fashionmnist_train), len(fashionmnist_test)

(60000, 10000)

In [5]:
# As for one sample, channel = 1, height, weight = 96
# for adapting to the input shape of googlenet
fashionmnist_train[0][0].shape, fashionmnist_test[0][0].shape

((1, 96, 96), (1, 96, 96))

In [6]:
train_loader = paddle.io.DataLoader(fashionmnist_train, batch_size=256, shuffle=True)
test_loader = paddle.io.DataLoader(fashionmnist_test, batch_size=64, shuffle=False)

### Model

In [7]:
import paddle.nn as nn
import paddle.nn.functional as F

# define inception block as googlenet's buliding blocks
class Inception(nn.Layer):

    # `c1`--`c4` are the number of output channels for each path
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # Path 1 is a single 1 x 1 convolutional layer
        self.p1_1 = nn.Conv2D(in_channels, c1, kernel_size=1)
        # Path 2 is a 1 x 1 convolutional layer followed by a 3 x 3
        # convolutional layer
        self.p2_1 = nn.Conv2D(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2D(c2[0], c2[1], kernel_size=3, padding=1)
        # Path 3 is a 1 x 1 convolutional layer followed by a 5 x 5
        # convolutional layer
        self.p3_1 = nn.Conv2D(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2D(c3[0], c3[1], kernel_size=5, padding=2)
        # Path 4 is a 3 x 3 maximum pooling layer followed by a 1 x 1
        # convolutional layer
        self.p4_1 = nn.MaxPool2D(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2D(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # Concatenate the outputs on the channel dimension
        return paddle.concat((p1, p2, p3, p4), axis=1)

In [8]:
b1 = nn.Sequential(nn.Conv2D(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2D(kernel_size=3, stride=2, padding=1))

W0307 20:06:18.511240 12954 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.0, Runtime API Version: 10.1
W0307 20:06:18.516088 12954 device_context.cc:465] device: 0, cuDNN Version: 7.6.


In [9]:
b2 = nn.Sequential(nn.Conv2D(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2D(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2D(kernel_size=3, stride=2, padding=1))

In [10]:
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2D(kernel_size=3, stride=2, padding=1))

In [11]:
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2D(kernel_size=3, stride=2, padding=1))

In [12]:
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2D((1,1)),
                   nn.Flatten()) 

In [13]:
googlenet = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))
print(googlenet)

Sequential(
  (0): Sequential(
    (0): Conv2D(1, 64, kernel_size=[7, 7], stride=[2, 2], padding=3, data_format=NCHW)
    (1): ReLU()
    (2): MaxPool2D(kernel_size=3, stride=2, padding=1)
  )
  (1): Sequential(
    (0): Conv2D(64, 64, kernel_size=[1, 1], data_format=NCHW)
    (1): ReLU()
    (2): Conv2D(64, 192, kernel_size=[3, 3], padding=1, data_format=NCHW)
    (3): ReLU()
    (4): MaxPool2D(kernel_size=3, stride=2, padding=1)
  )
  (2): Sequential(
    (0): Inception(
      (p1_1): Conv2D(192, 64, kernel_size=[1, 1], data_format=NCHW)
      (p2_1): Conv2D(192, 96, kernel_size=[1, 1], data_format=NCHW)
      (p2_2): Conv2D(96, 128, kernel_size=[3, 3], padding=1, data_format=NCHW)
      (p3_1): Conv2D(192, 16, kernel_size=[1, 1], data_format=NCHW)
      (p3_2): Conv2D(16, 32, kernel_size=[5, 5], padding=2, data_format=NCHW)
      (p4_1): MaxPool2D(kernel_size=3, stride=1, padding=1)
      (p4_2): Conv2D(192, 32, kernel_size=[1, 1], data_format=NCHW)
    )
    (1): Inception(
      (

In [14]:
# instantiation
model = paddle.Model(googlenet)

# visualization of the model workflow
model.summary((-1, 1, 96, 96)) # [N C H W]

-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
     Conv2D-1         [[1, 1, 96, 96]]     [1, 64, 48, 48]         3,200     
      ReLU-1         [[1, 64, 48, 48]]     [1, 64, 48, 48]           0       
    MaxPool2D-1      [[1, 64, 48, 48]]     [1, 64, 24, 24]           0       
     Conv2D-2        [[1, 64, 24, 24]]     [1, 64, 24, 24]         4,160     
      ReLU-2         [[1, 64, 24, 24]]     [1, 64, 24, 24]           0       
     Conv2D-3        [[1, 64, 24, 24]]     [1, 192, 24, 24]       110,784    
      ReLU-3         [[1, 192, 24, 24]]    [1, 192, 24, 24]          0       
    MaxPool2D-2      [[1, 192, 24, 24]]    [1, 192, 12, 12]          0       
     Conv2D-4        [[1, 192, 12, 12]]    [1, 64, 12, 12]        12,352     
     Conv2D-5        [[1, 192, 12, 12]]    [1, 96, 12, 12]        18,528     
     Conv2D-6        [[1, 96, 12, 12]]     [1, 128, 12, 12]   

{'total_params': 5977530, 'trainable_params': 5977530}

### Training

In [15]:
# optimizer and loss
model.prepare(optimizer = paddle.optimizer.Adam(parameters=model.parameters(), learning_rate=1e-3),
              loss=nn.CrossEntropyLoss(),
              metrics=paddle.metric.Accuracy())

# training
model.fit(train_loader,
        # eval_data = test_loader
        epochs=15,
        verbose=1,
        )

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/15


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 235/235 [==============================] - loss: 0.4162 - acc: 0.6761 - 84ms/step          
Epoch 2/15
step 235/235 [==============================] - loss: 0.3571 - acc: 0.8709 - 84ms/step          
Epoch 3/15
step 235/235 [==============================] - loss: 0.2894 - acc: 0.8974 - 104ms/step          
Epoch 4/15
step 235/235 [==============================] - loss: 0.1814 - acc: 0.9130 - 95ms/step          
Epoch 5/15
step 235/235 [==============================] - loss: 0.1584 - acc: 0.9223 - 85ms/step          
Epoch 6/15
step 235/235 [==============================] - loss: 0.1916 - acc: 0.9291 - 87ms/step          
Epoch 7/15
step 235/235 [==============================] - loss: 0.2276 - acc: 0.9346 - 84ms/step          
Epoch 8/15
step 235/235 [==============================] - loss: 0.1568 - acc: 0.9383 - 85ms/step          
Epoch 9/15
step 235/235 [==============================] - loss: 0.0769 - acc: 0.9447 - 85ms/step          
Epoch 10/15
step 235/235 [============

In [16]:
model.evaluate(fashionmnist_test, verbose=1)

Eval begin...
step 10000/10000 [==============================] - loss: 0.2357 - acc: 0.9275 - 15ms/step          
Eval samples: 10000


{'loss': [0.23569219], 'acc': 0.9275}